In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [39]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [40]:
df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoder=onehot_encoder_geo.fit_transform(df[['Geography']])

geo_column_names=onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df=pd.DataFrame(data=geo_encoder,columns=geo_column_names)

df=pd.concat([df.drop(columns='Geography',axis=1),geo_encoded_df],axis=1)

X=df.drop(columns=["Exited"],axis=1)
y=df['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file) 
    
with open ("scaler.pkl",'wb') as file:
    pickle.dump(scaler,file)


### Define a function to create the model and try different parameters (KerasClassifier)

In [41]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation="relu",input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation="relu"))

    model.add(Dense(1,activation="sigmoid"))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

    return model

## Create a Keras Classifier

In [42]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=1)

In [43]:
## Define the grid search parameters
param_grid={
    "neurons": [16,32,64,128],
    "layers": [1,2],
    "epochs": [50]
}

In [44]:
## Perform grid Search
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3,verbose=1)
grid_result=grid.fit(X_train,y_train)

# Print the best parameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\PYTHON\Python_Udemy\Projects\ANN Classification\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/50


800/800 [==============================] - 1s 862us/step - loss: 0.4679 - accuracy: 0.8014
Epoch 2/50
800/800 [==============================] - 1s 878us/step - loss: 0.3905 - accuracy: 0.8376
Epoch 3/50
800/800 [==============================] - 1s 861us/step - loss: 0.3627 - accuracy: 0.8529
Epoch 4/50
800/800 [==============================] - 1s 849us/step - loss: 0.3513 - accuracy: 0.8564
Epoch 5/50
800/800 [==============================] - 1s 844us/step - loss: 0.3472 - accuracy: 0.8565
Epoch 6/50
800/800 [==============================] - 1s 854us/step - loss: 0.3429 - accuracy: 0.8593
Epoch 7/50
800/800 [==============================] - 1s 882us/step - loss: 0.3391 - accuracy: 0.8609
Epoch 8/50
800/800 [==============================] - 1s 861us/step - loss: 0.3386 - accuracy: 0.8612
Epoch 9/50
800/800 [==============================] - 1s 862us/step - loss: 0.3368 - accuracy: 0.8602
Epoch 10/50
800/800 [==============================] - 1s 879us/step - loss: 0

## Best: 0.8567496204759851 using {'epochs': 50, 'layers': 2, 'neurons': 16}